In [3]:
from types import SimpleNamespace
from pprint import pprint
import numpy as np

import anomapy.train.sssn as sssn
import datasets

import pyworld.toolkit.tools.wbutils as wbu
import pyworld.toolkit.tools.torchutils as tu
import pyworld.toolkit.tools.visutils.jupyter as J
import pyworld.toolkit.tools.visutils.transform as T
import pyworld.toolkit.tools.visutils.plot as vplot
import pyworld.toolkit.tools.datautils.timeseries as TS

from scipy import fftpack

num_episodes = 1

envs = ['BeamRider','Breakout','Enduro','Pong','Qbert','Seaquest','SpaceInvaders']

def load_model(run):
    models, config = wbu.load(run) #load the most recently trained model
    model = models['model.pt'].load(sssn.model(**config))
    config = SimpleNamespace(**config)
    return model, config

def load_data(num_episodes):
    dataset = datasets.dataset('aad.clean.{0}'.format(env))
    dataset.state.transform.to_float().CHW().torch()
    return [d for d in dataset.state.load(num_episodes)]

def distance(z1, z2):
    return ((z1 - z2) ** 2).sum(1)

def spectrum(x, y):
    yf = fftpack.fft(y, x.size)
    amp = np.abs(yf) # get amplitude spectrum 
    freq = fftpack.fftfreq(x.size, max(x)/x.size)
    return freq[0:freq.size//2], (2/amp.size)*amp[0:amp.size//2]


In [4]:

num_episodes = 1
env = envs[4]

dryruns = sorted([r for r in wbu.dryruns() if env in r])
run = dryruns[0]
plot_path = "wandb/{0}/media/normal_hist-{1}.png".format(run, env)
model, config = load_model(run)
episodes = load_data(num_episodes)
episode = episodes[-1]

z = tu.to_numpy(sssn.encode(model, episode))

max_skip = int(z.shape[0]*.8)
def stat(i):
    d = distance(z[:-i], z[i:])
    return np.mean(d), np.min(d), np.max(d), np.var(d), i
mu, _min, _max, sig, x = zip(*[stat(i) for i in range(1,max_skip)])
mu, _min, _max, sig, x = np.array(mu), np.array(_min), np.array(_max), np.array(sig), np.array(x)
plot = J.plot([x,x,x], [mu,_min,_max], legend=['mean','min','max'])
plot.fig.update_layout(margin=dict(t=10,l=10,r=10,b=10), autosize=False, height=300, width=800, showlegend=False)
#plot.fig.write_image("wandb/{0}/media/skip_distance-{1}.png".format(run, env))
#J.plot(x, sig, legend=['var'])

x, dmu = TS.detrend(x, mu)
J.plot(x, dmu)
freq, amp = TS.spectrum(dmu)
wav = 1 / freq

J.plot(wav, amp)
w = int(wav[np.argmax(amp)]) #~ wave length of the signal



 -- found local run at /home/ben/Documents/repos/anomaly-detection/wandb/dryrun-20200429_171611-sssn-Qbert-20200429181610
 -- found config file: 
 -- found 1 model(s): 
 ---- model.pt
loading dataset aad.clean.Qbert.state...


IntProgress(value=0, max=1)

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
 …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': '1',
              'type': 'scatter',
    …

<ipython-input-4-d5c69cf5700b>:27: RuntimeWarning:

divide by zero encountered in true_divide



FigureWidget({
    'data': [{'mode': 'lines',
              'name': '1',
              'type': 'scatter',
    …

In [5]:
episode = T.HWC(tu.to_numpy(episodes[-1]))

de = episode[:-w] - episode[w:]
J.images(de)


de = episode[:-1] - episode[1:]
J.images(de)

    

(210, 160, 3)


interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=1019), Output()), _d…

Canvas(height=210, width=160)

(210, 160, 3)


interactive(children=(IntSlider(value=0, description='x', layout=Layout(width='99%'), max=1079), Output()), _d…

Canvas(height=210, width=160)

In [7]:
# PATH DISPLACEMENT IN PIXEL SPACE

num_episodes = 1


for env in envs:
    dryruns = sorted([r for r in wbu.dryruns() if env in r])
    run = dryruns[0]
    model, config = load_model(run)
    episodes = load_data(num_episodes)
    episode = episodes[-1]
    print(env)
    z = tu.to_numpy(episode).reshape(episode.shape[0],-1)
    max_skip = int(z.shape[0]*.8)
    def stat(i):
        #if not i % 200:
        #    print("---", i)
        d = distance(z[:-i], z[i:])
        return np.mean(d), np.min(d), np.max(d), np.var(d), i
    mu, _min, _max, sig, x = zip(*[stat(i) for i in range(1,max_skip)])
    mu, _min, _max, sig, x = np.array(mu), np.array(_min), np.array(_max), np.array(sig), np.array(x)
    plot = J.plot([x,x,x], [mu,_min,_max], legend=['mean','min','max'])
    plot.fig.update_layout(margin=dict(t=10,l=10,r=10,b=10), autosize=False, height=300, width=800, showlegend=False)
    plot.fig.write_image("wandb/{0}/media/pixel-displacement-{1}.png".format(run, env))
    
#J.plot(x, sig, legend=['var'])

 -- found local run at /home/ben/Documents/repos/anomaly-detection/wandb/dryrun-20200429_141220-sssn-BeamRider-20200429151217
 -- found config file: 
 -- found 1 model(s): 
 ---- model.pt
loading dataset aad.clean.BeamRider.state...


IntProgress(value=0, max=1)

BeamRider


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
 …

 -- found local run at /home/ben/Documents/repos/anomaly-detection/wandb/dryrun-20200429_150221-sssn-Breakout-20200429160219
 -- found config file: 
 -- found 1 model(s): 
 ---- model.pt
loading dataset aad.clean.Breakout.state...


...

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
 …

 -- found local run at /home/ben/Documents/repos/anomaly-detection/wandb/dryrun-20200501_093836-sssn-SpaceInvaders-20200501103833
 -- found config file: 
 -- found 1 model(s): 
 ---- model.pt
loading dataset aad.clean.SpaceInvaders.state...


IntProgress(value=0, max=1)

SpaceInvaders


FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
 …

# Spectrum Example

In [ ]:
#Import the necessary packages
from scipy import fftpack
import matplotlib.pyplot as plt
import numpy as np
    
# sampling freq in herts 20Hz, and 40Hz
freq_sampling1 = 10
freq_sampling2 = 20
amplitude1 = 2 # amplitude of first sine wave
amplitude2 = 4 # amplitude of second sine wave
time = np.linspace(0, 6, 500, endpoint=True) # time range with total samples of 500 from 0 to 6 with time interval equals 6/500
y = amplitude1*np.sin(2*np.pi*freq_sampling1*time) + amplitude2*np.sin(2*np.pi*freq_sampling2*time)
y += np.linspace(0.1,10,y.size) ** 2

y = y[1:] - y[:-1] # remove trend via differencing
time = time[1:]

plt.figure(figsize=(10, 4))
plt.plot(time,y, 'k', lw=0.8)
plt.xlim(0,6)
plt.show()

freq, amp = spectrum(time, y)

plt.figure(figsize=(10,6))
plt.plot(freq, amp)
plt.show()

#https://stackoverflow.com/questions/52730753/how-to-get-frequency-axis-from-an-fft-function

In [33]:
import numpy as np

x = np.linspace(0,50*np.pi,10000)

y1 = np.linspace(-2,2,x.size-5000)**2 + np.sin(x[:-5000])
y2 = np.linspace(-2,2,5000)
print(y1.shape, y2.shape)
y1 = np.concatenate((y1,y2))
#y2 = np.sin(x)
y = y1


def plot_skip_distance(z):
    max_skip = int(z.shape[0]*.8)
    def stat(i):
        d = distance(z[:-i], z[i:])
        return np.mean(d), np.min(d), np.max(d), np.var(d), i
    mu, _min, _max, sig, x = zip(*[stat(i) for i in range(1,max_skip)])
    mu, _min, _max, sig, x = np.array(mu), np.array(_min), np.array(_max), np.array(sig), np.array(x)
    J.plot([x,x,x], [mu,_min,_max], legend=['mean','min','max'])
    
#J.plot([x,x,x],[y1,y2,y])
J.plot(x,y)
plot_skip_distance(y[:,np.newaxis])

(5000,) (5000,)


FigureWidget({
    'data': [{'mode': 'lines',
              'name': '1',
              'type': 'scatter',
    …

FigureWidget({
    'data': [{'mode': 'lines',
              'name': 'mean',
              'type': 'scatter',
 …